# Correction Lab Word Embedding n°3

In [142]:
import LangueFr as FR # LangueFr est le Lab n°2
from tqdm import tqdm
import random,os

### Charger les Textes ( Tirés du livre "Candide" de Voltaire)

In [148]:
chapitres_name_list = [f[:-4] for f in os.listdir('Livres\\Candide\\') if f.endswith('.txt')]
chapitres_list = [open("Livres\\Candide\\"+f+'.txt','r',encoding='utf_8').read() for f in chapitres_name_list]

### Transformer le Texte en une liste de Phrase

In [149]:
stop_chars = ['.',';','?','!']

In [150]:
def ChapToList(chapitre):
    phrases = []
    i = 0
    for k,c in enumerate(chapitre):
        if c in stop_chars:
            phrases.append(chapitre[i:k])
            i = k+1
    return phrases

### Transformer les phrases en listes de mot

In [151]:
def PhraseToList(phrase):
    phrlist = [
        [''.join([c for c in word if c.isalpha() or c in ["'","-"]]) for word in sentence.split(' ')] for sentence in phrase
    ]
    phrlist = [[word.lower() for word in words if word != ''] for words in phrlist]
    return phrlist

In [152]:
phrases = []
for chapitre in chapitres_list:
    phrases.extend(PhraseToList(ChapToList(chapitre)))
print(len(phrases))

2146


## Charger le dictionnaire

In [153]:
import json

In [154]:
dictionnaire = js = json.load(open('dictionnary.json','r'))

In [155]:
dictionnaire[0]

{'M': {'mot': 'a', 'no': '1'},
 'CONT': 'tracer N',
 'DOM': {'nom': 'écriture, écrits'},
 'OP': 'lett',
 'SENS': 'alphabet latin',
 'OP1': 'R3a1',
 'CA': {'categorie': 'N', 'type': 'non-anime', 'genre': 'M'}}

In [156]:
# Format du dictionnaire
element = """
{
    'M':{"mot":%%,...}, # Le mot
    'CONT':?,
    'DOM':{'nom':%%},   # Domaine, Sujet, ...
    'OP':?,
    'SENS':%%,          # Sens du mot
    'OP1':?,
    'CA':{
        'categorie':%%, # Categorie : Nom, Verbe, Préposition, ...
        'type':%%,      # Type : Humain, nom-animé, ...
        'genre':%%,     # Genre : Masculin, Feminin
    }
}
"""

### Catégories des mots

In [157]:
categories = set([v['CA']['categorie'] for v in js])
categories

{'A', 'Adv', 'Conj', 'Interj', 'N', 'Vi', 'Vp', 'Vt'}

In [158]:
# A : ?
# Adv : Adverbe
# Conj : ?
# Interj : Interjection
# N : Nom
# Vi : Verbe intransitif
# Vt : Verbe Transitif
# Vp : Verbe ? 

### Vectoriser les mots selon le modèle du Lab n°2

#### Modéliser les noms

In [159]:
def strtog(g):
    if g == 'F':
        return FR.FEMININ
    elif g == 'M':
        return FR.MASCULIN
    return FR.SS_GENRE

In [160]:
# Liste de tous les mots
mots = [
    w['M']['mot'] for w in dictionnaire
]
# Liste de tous les verbes
all_verbes = [
    v for v in js if v['CA']['categorie'] in ['Vp','Vi','Vt']
    if not ' ' in v['M']['mot']
]
# Liste de tous les verbes du 1er groupe (Certain verbe ne sont pas vraiment du 1er groupe comme le verbe aller)
verbes_1er = [
    v['M']['mot'] for v in all_verbes 
    if v['M']['mot'].endswith('er') 
]

# Liste de tous les noms
Noms = [
    v for v in js if v['CA']['categorie'] == 'N'
]
# Dictionnaire des noms modélisés
Noms_modelise = {}

for nom in Noms:
    
    try:
        # Les noms qui finissent par "s" ou "x" ne s'accordent pas au pluriel
        # Les noms qui finissent par "al" s'accordent au pluriel en "aux" et au feminin en "alle" ( à part quelques exceptions)
        # Les règles sont nombreuses (feminin de nombreux :) ) ...
        
        if not nom['M']['mot'].endswith('s') and not nom['M']['mot'].endswith('x'):
            pluriel=FR.accord_s
        else:
            pluriel=None
        
        if not nom['M']['mot'].endswith('al'):
            pluriel=FR.accord_al
        
        # Modéliser 
        Noms_mods[nom['M']['mot']] = FR.mot(
            nom['M']['mot'],genre=strtog(nom['CA']['genre']),accord_pluriel=pluriel
        )
        
    except Exception as exc:
        pass

In [161]:
Noms_mods

{'a': a,
 'à-côté': à-côté,
 'à-coup': à-coup,
 'à-haut': à-haut,
 'a-module': a-module,
 'à-peu-près': à-peu-près,
 'à-pic': à-pic,
 'à-plat': à-plat,
 'à-propos': à-propos,
 'à-valoir': à-valoir,
 'à-venir': à-venir,
 'aa': aa,
 'aabam': aabam,
 'aalénien': aalénien,
 'abaca': abaca,
 'abacule': abacule,
 'abaisse': abaisse,
 'abaisse-langue': abaisse-langue,
 'abaisse-paupière': abaisse-paupière,
 'abaissement': abaissement,
 'abaisseur': abaisseur,
 'abajoue': abajoue,
 'abalé': abalé,
 'abalourdissement': abalourdissement,
 'abandon': abandon,
 'abandonnement': abandonnement,
 'abandonnisme': abandonnisme,
 'abaque': abaque,
 'abarco': abarco,
 'abasie': abasie,
 'abasourdissement': abasourdissement,
 'abat-carrage': abat-carrage,
 'abat-carre': abat-carre,
 'abat-feuille': abat-feuille,
 'abat-feuilles': abat-feuilles,
 'abat-foin': abat-foin,
 'abat-jour': abat-jour,
 'abat-son': abat-son,
 'abat-sons': abat-sons,
 'abat-vent': abat-vent,
 'abat-voix': abat-voix,
 'abatage': aba

#### Modéliser les verbes

In [162]:
verbes = {}

for v in verbes_1er:
    verbes[v]={
        'conjugaison':FR.conjugaison_1er,
        # On pourra après ajouter les verbes du 2ème groupe puis, après beaucoup de travail, les verbes du 3ème
    }

# Pre-Traitement des Phrases du texte avant Training du Modèle 

In [163]:
mots_associe = {}

In [164]:
personnes = [FR.JE,FR.TU,FR.IL,FR.NOUS,FR.VOUS,FR.ILS]
genres = [FR.MASCULIN,FR.FEMININ]

# Cette fonction retourne toutes les conjugaisons possibles pour un verbe donné
def formes_conjugaison(verbe):
    # Si on ne connait pas le mode de conjugaison du verbe : return []
    if verbes[verbe]['conjugaison'] is None:
        return []
    
    # Sinon on retourne la liste de toutes les conjugaisons du verbe à toutes les personnes avec tous les genres
    c = []
    for personne in personnes:
        c.append(repr(verbes[verbe]['conjugaison'](Labn2.mot(verbe),personne,genre,Labn2.PRESENT)))
    for personne in personnes:
        for genre in genres:
            c.append(repr(verbes[verbe]['conjugaison'](Labn2.mot(verbe),personne,genre,Labn2.PASSE_COMPOSE)))
      
    return c

In [165]:
# Afin d'éviter que notre modèle considère les deux mots "douce" et "douces" comme deux mots différents ou bien le verbe 
# "tomber" et sa conjugaison "tombons" comme deux mots différents, cette associe toute conjugaison à son verbe à l'infinitif
# et tous nom au pluriel au même nom au singulier

def AssocierMots(mot):
    
    try:
        # Si le mot a déjà été associé à un autre mot, il n'est pas nécessaire de refaire le traitement
        return mots_associe[mot]
    except:
        pass
    
    if len(mot) <= 3:
        return mot 
    
    if mot in verbes:
        return mot
    
    if mot in mots:
        return mot
    
    # Si le mot se termine par un "s" et qu'il n'est pas contenu dans notre dictionnaire, il y'a de fortes chance qu'en supprimant
    # le "s" on le trouve.
    if mot.endswith('s'):
        if mot[:-1] in mots:
            related[mot] = mot[:-1]
            return mot[:-1]
    # Si le mot se termine par un "e" et qu'il n'est pas contenu dans notre dictionnaire, il y'a de fortes chance qu'en supprimant
    # le "e" on le trouve.
    elif mot.endswith('e'):
        if mot[:-1] in mots:
            related[mot] = mot[:-1]
            return mot[:-1]
    
    # On cherche parmis toutes les conjugaisons possibles de tout les verbes commençant par la même lettre que le mot en question
    for v in [verbe for verbe in verbes if verbe.startswith(mot[0])]:
        if mot in formes_conjugaison(v):
            related[mot] = v
            return mot
    
    # Si rien n'a été trouvé on retourne lmot comme il est
    return mot
        

In [166]:
# Cette fonction condense le texte en remplaçant tout les mots par leur associés dans le dictionnaire en utilisant la
# fonction ci-dessus
def CondenserPhrase(phrase):
    phrase_condensee = []
    for w in phrase:
        try:
            phrase_condensee.append(AssocierMots(w))
        except Exception as exc:
            phrase_condensee.append(w)
    return phrase_condensee

In [167]:
phrases_condensees = []
with tqdm(total=len(phrases)) as pbar:
    for phrase in phrases:
        phrases_condensees.append(CondenserPhrase(phrase))
        pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 2146/2146 [01:39<00:00, 21.51it/s]


# Training des Modèles de Word Embedding

In [168]:
from gensim.models import Word2Vec
model_1 = Word2Vec(sentences=reconned_sentences, size=300, window=5, min_count=5, workers=4, sg=1)

In [169]:
model_1.wv.most_similar("baron")

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('château', 0.999718427658081),
 ('vestphalie', 0.9997139573097229),
 ('philosophe', 0.9997073411941528),
 ('thunder-ten-tronckh', 0.9996963143348694),
 ("l'abbé", 0.9996957182884216),
 ('coup', 0.9996906518936157),
 ('derrière', 0.9996898770332336),
 ('gouverneur', 0.9996894598007202),
 ('maîtresse', 0.9996844530105591),
 ('temps', 0.9996843338012695)]

In [170]:
from gensim.models import FastText
model_2 = FastText(sentences=reconned_sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [171]:
model_2.wv.most_similar("baron")

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('baronne', 0.9999240636825562),
 ('patron', 0.9998845458030701),
 ('thunder-ten-tronckh', 0.9998728036880493),
 ('gouverneur', 0.9998670816421509),
 ('environ', 0.9998663663864136),
 ('jésuite', 0.9998574256896973),
 ('monseigneur', 0.9998557567596436),
 ('monsieur', 0.999854564666748),
 ('condition', 0.9998505711555481),
 ('capitaine', 0.9998494386672974)]

# Ecriture du poème

In [172]:
dict1 = model_1.wv
dict2 = model_2.wv

In [175]:
# Cette fonction utilise les deux modèles dict1 et dict2 afin de construire une phrase à partir d'un mot (complement)
# La phrase est de la forme (Sujet Verbe Complement)
def GenererPhrase(complement):
    
    # Prendre tout les mots similaire à (complement) selon dict 1
    plus_proches = [w[0] for w in dict1.wv.most_similar(complement)]
    
    reccurs = 0;Limit = 5
    verbespp = []
    nomspp = []
    
    with tqdm(total=Limit) as pbar:
        
        while reccurs < Limit:
            
            # Selectionner les verbes parmis les mots similaires
            verbespp.extend([v for v in plus_proches if v in verbes])
            # Selectionner les noms parmis les mots similaires
            nomspp.extend([v for v in plus_proches if v in Noms_mods and not v in verbes and len(v) < 12 and len(v) > 5])
            
            reccurs += 1
            
            # Continuer récursivment à chercher parmis les mots proches
            temp = []
            for p in plus_proches:
                try:
                    temp.extend([w[0] for w in dict1.most_similar(p)])
                except:
                    temp.extend([w[0] for w in dict2.most_similar(p)])    
            plus_proches = temp
            
            # Si on trouve 5 verbes et 5 noms similaires à complement, on s'arrete de chercher
            if len(verbespp) > 5 and len(nomspp) > 5:
                break
                
            pbar.update(1)
    
    
    verbespp = list(set(verbespp))
    nomspp = list(set(nomspp))
    article = [FR.Un,FR.Deux]
    
    i = random.randint(0,len(verbespp)-1);verbespp=verbespp[i] # Choisir un verbe au hasard
    i = random.randint(0,len(nomspp)-1);nomspp=nomspp[i] # Choisir un nom au hasard
    i = random.randint(0,len(article)-1);article1=article[i] # Choisir un article au hasard (Pour construire le Sujet: article + nom)
    i = random.randint(0,len(article)-1);article2=article[i] # Choisir un article au hasard (Pour l'associer au complement: article + nom)
    
    return article1,nomspp,verbespp,article2,
    
    

In [181]:
Titre = 'portugal'
poeme = []

# Choisir un mot au hasrd parmi les mots similaires au titre
# De ce premier mot on va tirer la rime
rimes = [mot_[0] for mot_ in dict2.most_similar(Titre) if len(mot_[0]) < 10]
random.shuffle(rimes)
rime = rimes[0]

rimes = []
frimes = []
limit_poeme = 5 # Le poème est limité à 5 lignes

# Boucle de construction du poème
while len(poeme) < limit_poeme:

    try:
        # générer une phrase à partir du dernier mot (rime)
        nouvelle_ligne = list(GenererPhrase(rime))+[rime]
        poeme.append(nouvelle_ligne)
        
        rimes.append(rime)
        
    except Exception as exc:
        frimes.append(rime) # Frime contient les rime qui ont généré des erreurs (mots similaires non trouvé, ...)

    # Lister les condidats pour la prochaine rime
    nrime = [
        nms for nms in Noms_mods # Selectionner parmi tout les noms modélisé
        if not nms in rimes and  # Les mots qui n'ont pas encore été utilisé comme rime
        not nms in frimes and    # Qui n'ont pas générés d'erreur auparavant
        not nms in verbes and    # On cherche des nom pas des verbes
        nms.endswith(rime[-2:])  # Qui se terminent de la même manière que la rime actuelle
    ]

    try:
        # Selectionner une rime au hasard
        rime = nrime[random.randint(0,len(nrime)-1)]
    except:
        # Si la liste nrime est vide, on choisit de manière aléatoire la nouvelle rime (réinitialisation)
        primes = [rmes[0] for rmes in dict2.most_similar(Titre) if len(rmes[0]) < 10]
        random.shuffle(primes)
        rime = primes[0]

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
 40%|█████████████████████████████████▌                                                  | 2/5 [00:00<00:00, 11.77it/s]


## Bien conjuguer et accorder le poème

In [182]:
poeme

[[un, 'habitant', 'dîner', deux, 'ensuite'],
 [un, 'voyant', 'manger', deux, 'droite'],
 [un, 'raison', 'officier', un, 'reste'],
 [un, 'anglais', 'officier', un, 'pirate'],
 [deux, 'milieu', 'aller', un, 'ponte']]

In [184]:
# Mettre en bonne forme le poème
for line in poeme:
    try:
        nline = FR.Affirmation_Complementaire(
            FR.Nom_Commun(line[0],Noms_mods[line[1]]),
            FR.mot(line[2],conjugaison=verbes[line[2]]['conjugaison']),
            FR.Nom_Commun(line[3],Noms_mods[line[4]])
        )
        print(nline)
    except Exception as exc:
        pass
print()

un voyant mange deux droites
une raison officie un reste
un anglais officie un pirate
deux milieus allent une ponte



# FIN